In [23]:
!pip install python-dotenv

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [24]:
from dotenv import load_dotenv
import os
import requests
import csv
import time

In [ ]:
# Load the .env file
load_dotenv(dotenv_path = ".env")
# Access the variables
API_KEY = os.getenv('API_KEY')

In [20]:
def get_quarterly_total_assets(api_key, company, year, quarter):
    # Step 1: use the finnhub API to obtain "Financials As Reported"
    # Please pay attention to the API rate limit. I recommend you to let the program sleep for one second after you make an API call
    dataRequest = requests.get("https://finnhub.io/api/v1/stock/financials-reported", params= {'symbol': company,'freq': 'quarterly','token':api_key}).json()
    time.sleep(2)  # to respect the API rate limit

    # Step 2: parse the financials returned from the finnhub API to get the total assets in a specific year-quarter
    #Total assets is reported in the balance sheets ('bs'). Total assets of a company is the value of the balance sheet item with its concept == "us-gaap_Assets"
    #Also, a few exceptions may occur in this process:
    # (1) 'data' is not a key in the returned dictionary;
    # (2) the API does not return data for the specific year-quarter being queried;
    # (3) the financial data in the specific year-quarter does not have total assets (i.e., no balance sheet items have its concept == "us-gaap_Assets")
    # You can use if statements to handle these exceptions
    # Please listen to the lecture to get more details
    if 'data' in dataRequest:
        for report in dataRequest['data']:
            if report["year"] == year and report["quarter"] == quarter:
                for item in report['report']['bs']:
                    itemUpper = item['label'].upper()
                    if item['concept'] == "us-gaap_Assets" and ((item['label'] == "TOTAL ASSETS") or item['label'] == "Total Assets" or item['label'] == "Total assets" or item['label'] == "total assets"):
                        return item['value']
                    elif item['concept'] == "us-gaap_Assets" and item['label'].upper() == "ASSETS" and len(item['unit']) > 5: # This line was added because I noticed that a few entries in the AAPL output were using assets instead of total assets for some reason.
                        return item['value']
            else:
                continue
    else:
        return None


In [ ]:
def get_peers_quarterly_total_assets(api_key, company, year, quarter):
  # Step 1: use the finnhub API to obtain the subIndustry peers of a focal company
  peers = requests.get("https://finnhub.io/api/v1/stock/peers", params= {'symbol':company,'grouping':'subIndustry','token':api_key}).json()
  for peer in peers:
      #print(peer)

      # Step 2: for every peer company (including the focal company in the peers returned by the finnhub API)
      # we obtain its total assets in a specific year-quarter by calling the get_quarterly_total_assets function
      total_assets_data = get_quarterly_total_assets(api_key, peer, year, quarter)

      # Step 3: write the peer, its total assets in a specific year-quarter, year and quarter to a csv file. The csv file is named with {focal company name}.csv
      # For Step 3: learn how to write rows to a csv file from https://www.freecodecamp.org/news/how-to-create-a-csv-file-in-python/
      # For Step 3: you do not need to write a header in the csv file. Please see Canvas for example output files
      with open(f"./Assignment2/output/{company}.csv", mode='a', newline='') as file:
          writer = csv.writer(file)
          writer.writerow([peer, total_assets_data, year, quarter])
          file.close()


In [ ]:
# combine results from all requests
with open('./Assignment2/company_symbols.csv', 'r') as f:
    # loop through each row
    # the delimiter is comma
    for line in f:
      focal_company = line.strip() # each line read from the csv file includes a newline character "\n" at the end, so we use .strip() to strip the newline character from the end of the line

      # call the function get_peers_quarterly_total_assets to obtain the total assets of the focal company and its peers in a specific year-quarter (2025-Q1 for this assignment)
      get_peers_quarterly_total_assets(api_key = API_KEY,
                                  company = focal_company,
                                  year = 2025,
                                  quarter = 1)
